In [ ]:
# Perform necessary imports
from dotenv import load_dotenv
import os
import requests
import json
from datetime import datetime
import time

# Load variables from .env file
load_dotenv()

In [ ]:
# Read environment variables
BASE_URL = os.getenv("BASE_URL")
OAUTH_ENDPOINT = os.getenv("OAUTH_ENDPOINT")
ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")
CLIENT_ID = os.getenv("CLIENT_ID")
REFRESH_TOKEN = os.getenv("REFRESH_TOKEN")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
AUTHORIZATION_CODE = os.getenv("AUTHORIZATION_CODE")

In [ ]:
# Fetch the basic user profile of the user
athlete_profile = f'{BASE_URL}/athlete'
headers = {
    'Authorization': f'Bearer {ACCESS_TOKEN}'
}

response = requests.get(athlete_profile, headers=headers)

if response.status_code == 200:
    athelete_basic_data = response.json()
    print(json.dumps(athelete_basic_data, indent=4))
else:
    print("Error: ", response)

In [ ]:
# Perform OAuth authentication to generate access tokens for reading activities
params = {
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
    'code': AUTHORIZATION_CODE,
    'grant_type': 'authorization_code'
}

authentication_response = requests.post(OAUTH_ENDPOINT, params=params)

if authentication_response.status_code == 200:
    authentication_response_data = authentication_response.json()
    print(json.dumps(authentication_response_data, indent=4))
else:
    print("Error: ", authentication_response)

In [ ]:
# We use the access token from authentication_response_data to read user's activities
data_access_token = authentication_response_data['access_token']
print(data_access_token)

In [ ]:
# Fetch a list of user's activities using the retrieved access token
activities_list_endpoint = f'{BASE_URL}/athlete/activities'
before_ts = int(time.time())
after_ts = datetime(2023,1,1,0,0,1)
params = {
    'before': before_ts,
    'after': after_ts,
    'page': 1
}

headers = {
    'Authorization': f'Bearer {data_access_token}'
}

activities_response = requests.get(activities_list_endpoint, headers=headers, params=params)

if activities_response.status_code == 200:
    activites_response_data = activities_response.json()
    print(json.dumps(activites_response_data, indent=4))
else:
    print("Error: ", activities_response)

# dump the activities record into a JSON flatfile
with open('activity_data.json', 'w') as file:
    json.dump(activites_response_data, file, indent=4)
